## <center>Разметки новостной ленты </center>

Для улучшения предсказания временных рядов, курса валют или акций, полезно прибегнуть к ансамблю моделей, одна из которцх построена на LSTM (пример https://github.com/BlackStoneShadow/Python/blob/main/MLearning/ANN/lesson8/Task1.ipynb), другая учитывает новостной фон (https://github.com/BlackStoneShadow/Python/tree/main/MLearning/Markup/Lesson5/Task1 - реализация получение нововстной ленты), такая совместная работа моделей позволит выполнять более взвешенные предсказания значений.

In [365]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [366]:
pip install scikit-learn

In [367]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [368]:
pip install TextBlob

In [441]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from textblob import TextBlob

import pandas as pd
import numpy as np
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Protorsky\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [606]:
positive = ['рост', 'стабильность', 'здорово', 'надежда', 'победа', 'подорожали', 'высокая доходность']
negative = ['падение', 'кризис', 'рецессия', 'давление', 'беспокойство', 'приостановлены', 'арест', 'инфляционные ожидания']

def get_class(text):
    words = nltk.word_tokenize(text.lower())
    
    count_pos = sum([1 for word in words if word in positive])
    count_neg = sum([1 for word in words if word in negative])
    
    if count_pos > count_neg:
        return 'Positive'
    elif count_pos < count_neg:
        return 'Negative'
    else:
        return 'Neutral'

In [607]:
print(get_class('рост рынка'))
print(get_class('падение рынка'))
print(get_class('прогноз рынка'))

Positive
Negative
Neutral


In [608]:
def get_polarity(text): 
    blob = TextBlob(text) 
    return blob.sentiment.polarity

In [626]:
# механизм полярности не работает
print(get_polarity('рост рынка'))
print(get_polarity('падение рынка'))
print(get_polarity('прогноз рынка'))

0.0
0.0
0.0


In [627]:
news = pd.read_csv('news.csv', engine='python', encoding='utf-8', error_bad_lines=False)

news.head(10)

,date,article,text
0,2024-04-08 00:00:00,Борьба Мосбиржи с «несправедливыми IPO»: новос...,Investing.com — Мосбиржа внедрит сервис для бо...
1,2024-04-08 00:00:00,Главные новости: самолеты Boeing продолжают ра...,Investing.com — На этой неделе начнется новый ...
2,2024-04-08 00:00:00,«Конгрессу надо действовать быстро»: эксперт п...,Investing.com — Из-за высокого давления госдол...
3,2024-04-08 00:00:00,"США пытаются наладить отношения с Китам, но ес...",Investing.com — Министр финансов США Джанет Йе...
4,2024-04-09 00:00:00,Взрывной рост Toncoin: новости к утру 9 апреля,Investing.com — Toncoin обновил исторический м...
5,2024-04-09 00:00:00,Инфляционные ожидания в США стабилизируются,Investing.com — Ожидания американских потребит...
6,2024-04-09 00:00:00,Главные новости: выступления спикеров ФРС и но...,Investing.com — В преддверии выхода ключевых д...
7,2024-03-31 00:00:00,Топ-5 событий на этой неделе: экономданные из ...,Investing.com — Данные по занятости в США в пя...
8,2024-03-17 00:00:00,Топ-5 недели: в центре внимания - решения цент...,Investing.com — Решения центральных банков буд...
9,2024-03-18 00:00:00,Явная победа Путина на выборах: новости к утру...,"Investing.com — Путин набирает 87,33% на выбор..."


In [628]:
# выборка для выполнения разметки
train_news = news.sample(frac = 0.1)
print(f'length train={len(train_news)}')
# перемешиваем для сохранения распределения
train_news = shuffle(train_news)
# разметка на основе своих правил
train_news['sentiment_label'] = train_news['text'].apply(get_class)

length train=100


In [629]:
def train_model(labeled_news):     
    vectorizer = TfidfVectorizer()
    # векторизация текстовых данных с помощью TF-IDF 
    X = vectorizer.fit_transform(labeled_news['text']) 
    y = labeled_news['sentiment']
    # Обучение модели логистической регрессии на размеченных данных 
    model = LogisticRegression() 
    model.fit(X, y)
    
    return model, vectorizer

In [630]:
# выполним label-encoding
encoder = LabelEncoder().fit(train_news['sentiment_label'])
train_news['sentiment'] = encoder.transform(train_news['sentiment_label'])
# делим набор для начала активного обучения
labeled_news, unlabeled_news = train_test_split(train_news, train_size=0.2, random_state=42)
# обучение модели
model, vectorizer = train_model(labeled_news)
# предсказание
X_unlabeled = vectorizer.transform(unlabeled_news['text']) 
y_unlabeled_predicted = model.predict(X_unlabeled)
# определение энтропии данных
y_unlabeled_proba = model.predict_proba(X_unlabeled) 
uncertainty = -(y_unlabeled_proba * np.log2(y_unlabeled_proba)).sum(axis=1)
# поиск записей требующих разметки человеком
labeled_news_new = unlabeled_news.iloc[uncertainty.argsort()[:20]][['date','article','text','sentiment_label']]
# требуется ручная разметка
labeled_news_new.to_csv('uncertainty.csv')
labeled_news_new.head(10)

,date,article,text,sentiment_label
114,2024-02-27 00:00:00,Йеллен оценила риски для доллара при конфискац...,Investing.com — Министр финансов США Джанет Йе...,Neutral
267,2024-02-06 00:00:00,Китай начнет производить чипы следующего покол...,Investing.com — Китайская собрала в Шанхае пр...,Neutral
191,2024-01-26 00:00:00,Генслер призвал Европу ускорить расчеты по акц...,Investing.com — По мнению главы Комиссии по це...,Neutral
150,2024-02-14 00:00:00,«Альфа-Капитал» потребует с Euroclear рекордну...,Investing.com — Управляющая компания «Альфа-Ка...,Neutral
80,2024-03-13 00:00:00,Опустошение Euroclear из-за исков России: ново...,Investing.com — Reuters сообщил о риске финанс...,Neutral
176,2024-02-08 00:00:00,Потребительские цены в Китае показали самый бо...,Investing.com — Потребительские цены за январь...,Negative
87,2024-04-03 00:00:00,Когда наступит рецессия в США?,Investing.com — Аналитики банка развеяли опас...,Negative
174,2024-02-20 00:00:00,Первая победа частного инвестора над Euroclear...,Investing.com — Арбитражный суд впервые удовле...,Positive
41,2024-03-29 00:00:00,Замедление инфляции не сбивает Банк Японии с п...,Investing.com — Согласно недавно опубликованны...,Positive
276,2023-12-27 00:00:00,Минфин: обмен заблокированных активов завершит...,Investing.com — В Министерстве финансов надеют...,Neutral


![Markup LabelStudio](./label.jpg "LabelStudio")

In [641]:
# загрузка после ручной разметки
labeled_news_new = pd.read_csv('labeled.csv', engine='python', encoding='utf-8', error_bad_lines=False)
labeled_news_new = labeled_news_new[['date','article','text','sentiment']]
# выполним label-encoding
labeled_news_new['sentiment_label'] = labeled_news_new['sentiment']
labeled_news_new['sentiment'] = encoder.transform(labeled_news_new['sentiment_label'])
# объединение наборов
labeled_news = pd.concat([labeled_news, labeled_news_new])
# не забываем перемешать
labeled_news = shuffle(labeled_news)
# делим набор на обучающий и тестовый
train_news, test_news = train_test_split(labeled_news, train_size=0.2, random_state=80)
# переобучение модели на полных данных
model, vectorizer = train_model(train_news)

In [642]:
# расчитываем метрику
X_test = vectorizer.transform(test_news['text']) 
y_test_predicted = model.predict(X_test) 

f1 = f1_score(np.array(test_news['sentiment']), y_test_predicted, average='macro')

print(f'f1={f1}\nметрика очень слабая - необходимо улучшение правил разметки и повышение количества строк обработанных человеком')

f1=0.5673234811165845
метрика очень слабая - необходимо улучшение правил разметки и повышение количества строк обработанных человеком


<center> поскольку у нас три класса используем averago='micro' </center>

![averago='micro'](./micro.png "averago=micro")

In [633]:
# формируем итоговый набор с предсказанными классами
X = vectorizer.transform(news['text']) 
y_predicted = model.predict(X) 

news = pd.DataFrame({'date': news['date'], 'article':news['article'], 'text':news['text'], 'sentiment_label': encoder.inverse_transform(y_predicted)})

news.head(10)

,date,article,text,sentiment_label
0,2024-04-08 00:00:00,Борьба Мосбиржи с «несправедливыми IPO»: новос...,Investing.com — Мосбиржа внедрит сервис для бо...,Positive
1,2024-04-08 00:00:00,Главные новости: самолеты Boeing продолжают ра...,Investing.com — На этой неделе начнется новый ...,Negative
2,2024-04-08 00:00:00,«Конгрессу надо действовать быстро»: эксперт п...,Investing.com — Из-за высокого давления госдол...,Negative
3,2024-04-08 00:00:00,"США пытаются наладить отношения с Китам, но ес...",Investing.com — Министр финансов США Джанет Йе...,Positive
4,2024-04-09 00:00:00,Взрывной рост Toncoin: новости к утру 9 апреля,Investing.com — Toncoin обновил исторический м...,Positive
5,2024-04-09 00:00:00,Инфляционные ожидания в США стабилизируются,Investing.com — Ожидания американских потребит...,Positive
6,2024-04-09 00:00:00,Главные новости: выступления спикеров ФРС и но...,Investing.com — В преддверии выхода ключевых д...,Positive
7,2024-03-31 00:00:00,Топ-5 событий на этой неделе: экономданные из ...,Investing.com — Данные по занятости в США в пя...,Negative
8,2024-03-17 00:00:00,Топ-5 недели: в центре внимания - решения цент...,Investing.com — Решения центральных банков буд...,Negative
9,2024-03-18 00:00:00,Явная победа Путина на выборах: новости к утру...,"Investing.com — Путин набирает 87,33% на выбор...",Positive


In [634]:
news.to_csv('result.csv')